This notebook can be used to predict a single test review against the each of the GRU-RNN models created per author in the RNN-GRU.ipynb notebook. The preprocessing and prediction code is identical to the aforementioned notebook. The test review can be selected below.

In [0]:
!pip install pickle
!pip install numpy
!pip install tensorflow==1.15.0

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [0]:
import pickle
import numpy as np
from tensorflow import keras

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
data_dir = 'gdrive/My Drive/Colab Notebooks/AuthorshipAttribution/data' # @param {type:"string"}
model_dir = 'gdrive/My Drive/Colab Notebooks/AuthorshipAttribution/models/gru_models' # @param {type:"string"}

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
test_data = data_dir + "/test_data.pickle"
with open(test_data, "rb") as f:
    test_data = pickle.load(f)

In [0]:
x_test = test_data['test_texts']
y_test = test_data['test_labels']

In [0]:
def get_chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

In [0]:
# 182 words is quite short
# Try to join 5 tests texts together
longer_test_texts = get_chunks(x_test, 5)
longer_test_labels = get_chunks(y_test, 5)

In [0]:
longer_test_texts = ['\n'.join(chunk) for chunk in longer_test_texts]
longer_test_labels = [chunk[0] for chunk in longer_test_labels]

In [0]:
author_models = []
for i in range(50):
    author_models.append((keras.models.load_model(model_dir + "/author"+str(i)+".h5"), i))

In [0]:
# vectorization - chars to ints
import string
import random
import sys

import numpy as np

from tensorflow import keras
# keras.models import load_model

def sample(preds, temperature=1.0):
    """Sample predictions from a probability array"""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-6) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate(model, diversity=0.5, text=""):
    """Generate text from a model"""
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(5000):
        x = np.zeros((1, maxlen), dtype=np.int)
        for t, char in enumerate(sentence):
            try:
                x[0, t] = char_indices[char]
            except:
                print(sentence)
        preds = model.predict(x, verbose=0)[0][0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    return

def vectorize(text):
    """Convert text into character sequences"""
    step = 3
    sentences = []
    next_chars = []
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    X = np.zeros((len(sentences), maxlen), dtype=np.int)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t] = char_indices[char]
        y[i, char_indices[next_chars[i]]] = 1
    return X, y

def clean_text(text, charset):
    text = " ".join(text.split())  # all white space is one space
    text = "".join([x for x in text if x in charset])  # remove characters that we don't care about
    return text

def get_model(modelfile, freeze=False):
    model = keras.models.load_model(modelfile)
    if freeze:
        for layer in model.layers[:6]:
            layer.trainable = False
    return model

chars = " " + string.ascii_letters + string.punctuation  # sorted to keep indices consistent
charset = set(chars)  # for lookup
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 100  # must match length which generated model - the sequence length


Select the test review here. Enter any number between 0 and 499.

In [0]:
#Edit this to predict for a candidate review in the corresponding index between 0 and 499
test_review_index =   100# @param {type:"integer"}

In [0]:
# indicies = list(range(len(longer)))
test_texts = np.array(longer_test_texts)
test_labels = np.array(longer_test_labels)

text = test_texts[test_review_index]
label = test_labels[test_review_index]
X, y = vectorize(clean_text(text, charset))


In [0]:
from datetime import datetime
predictions = []
losses = []
t1 = datetime.now()
for am in author_models:
    print(".", end="")
    model = am[0]
    label = am[1]
    loss = model.evaluate(X, y, verbose=0)
    losses.append((loss, label))
print(" {}".format(datetime.now() - t1))
predictions.append(losses)

.................................................. 0:04:51.856830


In [0]:
pred_is = []
for pred in predictions:
    pred_i = [p[0] for p in pred]
    pred_is.append(pred_i)
pred = [np.argmin(pred) for pred in pred_is]

See the predicted author id and actual author id below.

In [0]:
#predicted author id
pred

[10]

In [0]:
#actual author id
test_labels[test_review_index]

10